In [ ]:
import googleapiclient.discovery
from dotenv import load_dotenv
import pandas as pd
import os

In [ ]:
# NOTE: You need a youtube v3 api key for this to work
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [ ]:
# Load youtube api client
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

In [ ]:
# get comment by youtbe Id
def getcomments(video):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video,
        maxResults=100
    )

    comments = []

    # Execute the request.
    response = request.execute()

    # Get the comments from the response.
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        public = item['snippet']['isPublic']
        comments.append([
            comment['authorDisplayName'],
            comment['publishedAt'],
            comment['likeCount'],
            comment['textOriginal'],
            comment['videoId'],
            public
        ])

    while (1 == 1):
        try:
            nextPageToken = response['nextPageToken']
        except KeyError:
            break
        nextPageToken = response['nextPageToken']
        # Create a new request object with the next page token.
        nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
        # Execute the next request.
        response = nextRequest.execute()
        # Get the comments from the next response.
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            public = item['snippet']['isPublic']
            comments.append([
                comment['authorDisplayName'],
                comment['publishedAt'],
                comment['likeCount'],
                comment['textOriginal'],
                comment['videoId'],
                public
            ])

    df = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
    return df

In [ ]:
videoID = "BsdClG_V0Do" # get id from a youtube url
df = getcomments(videoID)

In [ ]:
df.to_csv(f"data/comments_{videoID}.csv", index=False)